# Cline Prompt Learning Optimization on SWE-bench

This notebook demonstrates how we used Prompt Learning to optimize Cline's performance on the SWE-bench dataset. Cline is a popular and powerful open-source coding agent. We look to improve its performance on SWE-bench by optimizing its **rules**, which are user specified instructions that Cline appends to its system prompt. 

[More on Cline](https://www.google.com/search?q=cline&sca_esv=764700c983d0c1df&sxsrf=AE3TifOTqxMNetNu45T7bn53deGE6bPn3w%3A1759280858717&ei=2n7caKLCK6XK0PEPpuCv6AY&ved=0ahUKEwiil5P154GQAxUlJTQIHSbwC20Q4dUDCBA&uact=5&oq=cline&gs_lp=Egxnd3Mtd2l6LXNlcnAiBWNsaW5lMgoQIxiABBgnGIoFMgoQIxiABBgnGIoFMgoQIxiABBgnGIoFMhMQLhiABBixAxjRAxhDGMcBGIoFMhMQLhiABBixAxjRAxgUGIcCGMcBMgoQABiABBhDGIoFMgoQABiABBhDGIoFMgUQABiABDIFEAAYgAQyChAAGIAEGEMYigVIkw1Q8QNYpgxwA3gBkAEAmAG2AaABkAKqAQMxLjG4AQPIAQD4AQGYAgWgApwCwgIKEAAYsAMY1gQYR8ICDRAAGIAEGLADGEMYigXCAg4QABiwAxjkAhjWBNgBAcICDhAuGLADGLgGGMgD2AEBwgIQEC4YgAQY0QMYQxjHARiKBZgDAIgGAZAGE7oGBggBEAEYCZIHAzQuMaAHwhiyBwMxLjG4B5QCwgcDMC41yAcJ&sclient=gws-wiz-serp#:~:text=Cline%20%2D%20AI%20Coding,https%3A//cline.bot)

[More on Prompt Learning](https://arize.com/blog/prompt-learning-using-english-feedback-to-optimize-llm-systems/)

## Plan Mode (for now)

This is a primitive stage of optimization. We are just looking at **Plan Mode** for Cline, which generates a plan for a given query, referencing the files in the codebase. We are then using an LLM-as-Judge evaluator to judge the generated plan. Therefore the accuracies should be taken lightly - they are not a perfect reflection of Cline's performance because Cline is not actually editing the codebase, and we're not actually running the SWE Bench tests to verify whether its edits are correct.

We are still working on running Cline in **Act Mode**, and allowing it actually edit the codebase. Then we can use the tests in SWE bench to compute a firm accuracy of whether Cline made the right edits. Stay tuned.

## Setup

**Please visit README.md and complete all the Setup before running this notebook!**


## Important Note

Running this notebook can be computationally intensive and expensive as it involves multiple API calls to Claude for each SWE-bench instance. Consider adjusting the training and test set sizes based on your requirements and budget constraints.


In [2]:
import json
import os
import random
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import pandas as pd
from swebench.harness.utils import load_swebench_dataset

# Add parent directory to Python path to find optimizer_sdk
notebook_dir = Path().absolute()  # Get the current notebook's directory
parent_dir = notebook_dir.parent  # Get the parent directory
sys.path.append(str(parent_dir))  # Add it to Python path

from constants import CLINE_PROMPT
from evals import evaluate_results
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer
from run_cline import run_cline

2025-09-30 16:02:25,770 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: /Users/priyanjindal/.phoenix
2025-09-30 16:02:25,778 - phoenix.inferences.inferences - INFO - Dataset: phoenix_inferences_f39c392d-f102-4b50-9f8a-a68ca1499ee9 initialized


## Configuration

- OPTIMIZATION_LOOPS: number of Prompt Learning loops. How many times you want to optimize your prompt. 
- TRAIN_SIZE: size of training set. 
- TEST_SIZE: size of test set.
- MAX_WORKERS: SWE Bench is set up to run in parallel, with however many workers you specify. Set this relative to your machine and your Claude rate limits. 
- RULES: base starting ruleset. I suggest keeping the rule regarding resume_task, as I've noticed using the resume_task tool leads to unstable behavior.

In [ ]:
OPTIMIZATION_LOOPS = 5
TRAIN_SIZE = 150 ## Lower this based on your Claude rate limits.
TEST_SIZE = 150 ## Lower this based on your Claude rate limits.
MAX_WORKERS = 50 ## Lower this based on your Claude rate limits + your machine's memory.
RULES = "do NOT use resume_task tool. Do NOT ask for user input/confirmation at any step of the process."

## Train/Test Datasets

This code splits SWE-Bench Lite into train/test splits. 

The train set will be used to optimize the ruleset, while the test set will be used to measure the success of optimized rulesets. 

In [ ]:
dataset = load_swebench_dataset("SWE-bench/SWE-bench_Lite", "test")
random.seed(42)
random.shuffle(dataset)
train_dataset = dataset[:TRAIN_SIZE]
test_dataset = dataset[len(dataset)-TEST_SIZE:]
train_pd = pd.DataFrame.from_dict(train_dataset)
test_pd = pd.DataFrame.from_dict(test_dataset)
train_pd.to_csv("train_dataset.csv")
test_pd.to_csv("test_dataset.csv")

This helper function will help us convert our Cline runs on SWE Bench into data we can evaluate.

In [ ]:
def collect_swebench_results(all_results):
    dataset_rows = []
    for result in all_results:
        instance_id = result["instance_id"]
    # Find the corresponding problem statement from train_dataset
    problem_statement = next(inst["problem_statement"] for inst in dataset if inst["instance_id"] == instance_id)
    patch = next(inst["patch"] for inst in dataset if inst["instance_id"] == instance_id)
    test_patch = next(inst["test_patch"] for inst in dataset if inst["instance_id"] == instance_id)
    if result["final_plan"]:
        dataset_rows.append({
            "instance_id": instance_id,
            "problem_statement": problem_statement,
            "final_plan": result["final_plan"],
            "test_patch": test_patch,
            "patch": patch,
        })
    train_df = pd.DataFrame(dataset_rows)
    return train_df

## Helper: Running Cline on a dataset

This helper function runs Cline on a dataset. It is meant to be used to run Cline on either your train or test split. 

It runs Cline in parallel, spinning up MAX_WORKERS # of Cline servers at a time, each server running on a specific row of SWE Bench. 

It also then evaluates the plans generated by Cline using our LLM-as-judge eval. We simply provide an LLM with the problem statement, the test patch, the ground truth patch, and Cline's generated plan, and ask it if the generated plan seems right. We use this to compute a rough measure of Cline's accuracy.

In [ ]:
def run_cline_dataset(dataset):
    all_results = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futs = {ex.submit(run_cline, inst, i, RULES): inst["instance_id"] for i, inst in enumerate(dataset)}
        for fut in as_completed(futs):
            r = fut.result()
            all_results.append(r)
            print(f"[{r['instance_id']}] done")
    
    df = collect_swebench_results(all_results)
    evaluated_results = evaluate_results(df)

    accuracy = sum(evaluated_results["correctness"] == "correct") / len(evaluated_results)
    return evaluated_results, accuracy

## Ruleset Optimization

This code optimizes our ruleset for Cline. Here are the steps:

Repeats OPTIMIZATION_LOOPS # of times:
1. Run Cline, with the current ruleset, on the training set, and compute training accuracy.
2. Run Cline, with the current ruleset, on the test set, and compute test accuracy.
3. Use the results on the training set to optimize the ruleset, using `PromptLearningOptimizer'
4. Update the current ruleset to be the optimized ruleset, for the next iteration.

In [ ]:
for idx in range(OPTIMIZATION_LOOPS):
    print(f"Running for idx: {idx}")

    evaluated_train_results, train_accuracy = run_cline_dataset(train_dataset)
    evaluated_train_results.to_csv(f"results/train_{idx}.csv")
    evaluated_test_results, test_accuracy = run_cline_dataset(test_dataset)
    evaluated_test_results.to_csv(f"results/test_{idx}.csv")
    
    pl_optimizer = PromptLearningOptimizer(
        prompt=CLINE_PROMPT,
        model_choice="gpt-4o",
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
    ruleset = pl_optimizer.optimize(
        dataset=evaluated_train_results,
        output_column="final_plan",
        feedback_columns=["correctness", "explanation"],
        ruleset = ruleset,
        context_size_k=100000
    )

    with open(f"rulesets/ruleset_{idx}.txt", "w") as f:
        f.write(f"train_accuracy: {train_accuracy}")
        f.write(f"test_accuracy: {test_accuracy}")
        f.write(f"optimized_ruleset_{idx}: {ruleset}")
        f.write(ruleset)